In [68]:
import osmnx as ox
from shapely.geometry import Polygon, MultiPolygon
import geopandas as gpd
import requests 
from collections import defaultdict
from ipynb.fs.full.transportationNetworks import bounding_coordinates, flatten_coords

In [71]:
def count_objects_in_polygon(polygon):
    if not isinstance(polygon, (Polygon, MultiPolygon)):
        raise TypeError("Not polygon obj")

    shop_types = ['grocery', 'general', 'supermarket', 'deli', 'greengrocer', 'food', 'marketplace', 'kiosk']
    

    grocery_stores = ox.features.features_from_polygon(polygon, tags={"shop":shop_types, "amenity":shop_types})
    
    shop_counts = defaultdict(int)
    
    # Iterate over each grocery store and increment the count for its shop type
    shop_counts = defaultdict(int)
    for shop_type in shop_types:
        num_shops = len(grocery_stores[(grocery_stores['shop'] == shop_type) | (grocery_stores['amenity'] == shop_type)])
        shop_counts[shop_type] = num_shops
    
    return dict(shop_counts)

In [79]:
def return_weight(city, country):
    poly_coords = bounding_coordinates(city, country)[0]
    poly_coords = flatten_coords(poly_coords)
    poly = Polygon(poly_coords)


    object_counts = count_objects_in_polygon(poly)
    square_surface_area = poly.area
    
    weightSum = object_counts['grocery']*3 + object_counts['general']*3 + object_counts['supermarket']*3 + object_counts['deli'] *2 + object_counts['greengrocer'] * 2 + object_counts['food'] * 1 + object_counts['marketplace']*3 + object_counts['kiosk']*1

    print(weightSum, square_surface_area)
    return weightSum/square_surface_area


In [80]:
def test():
    print(return_weight('Cidade De Chimoio', 'Mozambique'))
test()

24 0.01484637184041495
1616.5565740894988
